In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

def iqr(data):
    return data.quantile(0.75) - data.quantile(0.25)

# Load the entire JSON file
file_path = "data/dynamic_log_icra2024.json"
with open(file_path, 'r') as file:
    data = json.load(file)

# Create DataFrame
df = pd.json_normalize(data)

# Calculate ratios
df['Discoverable/Total Ratio'] = df['parameters.problem.n_discoverable'] / df['parameters.problem.n_total']
df['Visited/Total Ratio'] = df['result.n_visited'] / df['parameters.problem.n_total']
df['Total Time'] = df['result.solution_segments'].apply(lambda x: sum(seg['time'] for seg in x))
df['Total Path Length'] = df['result.solution_segments'].apply(lambda x: sum(seg['path_length'] for seg in x))

# Sort DataFrame by 'Discoverable/Total Ratio' and 'parameters.planner'
df.sort_values(['Discoverable/Total Ratio', 'parameters.planner'], inplace=True)

# Separate DataFrame by planner type
df_fre = df[df['parameters.planner'] == 'dynamic_planner_fre']
df_lci = df[df['parameters.planner'] == 'dynamic_planner_lci']

# Metrics to plot
metrics = ['Total Time', 'Total Path Length', 'Visited/Total Ratio']

# Extract unique tree models
tree_models = df['parameters.problem.tree_model'].unique()

# Create a 3xN grid of subplots (for 3 metrics and N tree models)
fig, axs = plt.subplots(len(metrics), len(tree_models), figsize=(15, 15))

# For each metric and tree model, create a plot
for i, metric in enumerate(metrics):
    for j, tree_model in enumerate(tree_models):
        # Filter data for the current tree model
        df_fre_tree = df_fre[df_fre['parameters.problem.tree_model'] == tree_model].groupby('Discoverable/Total Ratio')[metric]
        df_lci_tree = df_lci[df_lci['parameters.problem.tree_model'] == tree_model].groupby('Discoverable/Total Ratio')[metric]
        
        # Calculate means and IQRs
        fre_means = df_fre_tree.mean()
        fre_errors = df_fre_tree.apply(iqr)
        lci_means = df_lci_tree.mean()
        lci_errors = df_lci_tree.apply(iqr)
        
        # Create a line plot with error bars
        axs[i, j].errorbar(fre_means.index, fre_means.values, yerr=fre_errors.values, fmt='o-', label='FRE')
        axs[i, j].errorbar(lci_means.index, lci_means.values, yerr=lci_errors.values, fmt='x-', label='LCI')

        # Extract 'n_total' value (it should be same for all rows, so just use first one)
        n_total = df_fre[df_fre['parameters.problem.tree_model'] == tree_model]['parameters.problem.n_total'].iloc[0]
        
        # Set plot title (with 'n_total'), labels, and legend
        axs[i, j].set_title(f'{metric} for {tree_model} (n_total={n_total})')
        axs[i, j].set_xlabel('Discoverable/Total Ratio')
        axs[i, j].set_ylabel(metric)
        axs[i, j].legend()
        
        # Set y-axis limit to start at 0
        axs[i, j].set_ylim(bottom=0)

# Improve layout
plt.tight_layout()
plt.show()
